In [ ]:
import os
from utils import *
from agents import *
import time
import torch
import torch.nn as nn
from copy import deepcopy
import argparse

In [2]:
parser = argparse.ArgumentParser()
parser.add_argument('--seeds',          type=int,       default=[2023, 2024, 2025])
parser.add_argument('--dataset',        type=str,       default='cifar100')
parser.add_argument('--batch_size',     type=int,       default=512)
parser.add_argument('--model_name',     type=str,       default='resnet18')
parser.add_argument('--retrain',        type=bool,      default=False)
parser.add_argument('--unlearn_class',  type=list,      default=3)
args = parser.parse_args("")
args.time_str = time.strftime("%m-%d-%H-%M", time.localtime())
if args.dataset.lower() == 'fmnist':
    args.n_channels = 1
else:
    args.n_channels = 3

if args.dataset.lower() == 'cifar100':
    args.num_classes = 100
else:
    args.num_classes = 10
criterion = nn.CrossEntropyLoss()

In [9]:
dict = {
    10: {'unlearn_class': [65, 80, 51, 28, 85, 97, 67, 86, 19, 32],
        'arxiv_name': 'retrain_model_12-20-03-04'},

    20 : {'unlearn_class': [65, 80, 51, 28, 85, 97, 67, 86, 19, 32, 
                  11, 24, 56, 68, 90, 39, 42, 37, 89, 72],
        'arxiv_name': 'retrain_model_12-20-11-14'},

    30: {'unlearn_class': [65, 80, 51, 28, 85, 97, 67, 86, 19, 32, 
                  11, 24, 56, 68, 90, 39, 42, 37, 89, 72, 
                  71, 9,  48, 99, 49, 35, 3,  22, 12, 82],
        'arxiv_name': 'retrain_model_12-22-00-15'},

    40: {'unlearn_class': [65, 80, 51, 28, 85, 97, 67, 86, 19, 32, 
                        11, 24, 56, 68, 90, 39, 42, 37, 89, 72, 
                        71, 9, 48, 99, 49, 35, 3, 22, 12, 82, 
                        64, 61, 8, 79, 4, 60, 83, 66, 5, 23],
        'arxiv_name': 'retrain_model_12-22-01-28'},

    50: {'unlearn_class': [65, 80, 51, 28, 85, 97, 67, 86, 19, 32, 
                            11, 24, 56, 68, 90, 39, 42, 37, 89, 72, 
                            71, 9, 48, 99, 49, 35, 3, 22, 12, 82, 
                            64, 61, 8, 79, 4, 60, 83, 66, 5, 23, 
                            70, 33, 78, 13, 81, 43, 14, 40, 36, 34],
        'arxiv_name': 'retrain_model_12-22-02-49'}
}



In [6]:
train_loader, val_loader, test_loader = get_dataloader(args)
remain_train_loader, unlearn_train_loader = split_2_remain_unlearn(args, train_loader)
remain_val_loader, unlearn_val_loader = split_2_remain_unlearn(args, val_loader)
remain_test_loader, unlearn_test_loader = split_2_remain_unlearn(args, test_loader)

remain_class = np.setdiff1d(np.arange(args.num_classes), args.unlearn_class)
train_targets_list = np.array(train_loader.dataset.targets)[train_loader.sampler.indices]

Files already downloaded and verified
Files already downloaded and verified


# unlearn 10 classes

In [24]:
num_unlearn = 10
args.unlearn_class = dict[num_unlearn]['unlearn_class']
arxiv_name_r = dict[num_unlearn]['arxiv_name']

arxiv_name_o = 'original_model_12-20-05-11'
Acc_r, Acc_f = np.zeros((2,3)), np.zeros((2,3))
for i in range(3):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/{arxiv_name_o}_{args.seeds[i]}.pth'))
    Acc_f[0][i], Acc_r[0][i] = test_by_class(model, test_loader, i=args.unlearn_class)

print('------------ Retrained model ------------')
for i in range(3):
    model_r = get_model(args)
    model_r.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/{arxiv_name_r}_{args.seeds[i]}.pth'))
    Acc_f[1][i], Acc_r[1][i] = test_by_class(model_r, test_loader, i=args.unlearn_class)
  

print(f'Original model Acc_f: {100*Acc_f[0].mean():.2f} \pm {100*Acc_f[0].std():.2f}')
print(f'Original model Acc_r: {100*Acc_r[0].mean():.2f} \pm {100*Acc_r[0].std():.2f}')

print(f'Retrained model Acc_f: {100*Acc_f[1].mean():.2f} \pm {100*Acc_f[1].std():.2f}')
print(f'Retrained model Acc_r: {100*Acc_r[1].mean():.2f} \pm {100*Acc_r[1].std():.2f}')


0.9100, 0.8300, 0.6100, 0.6500, 0.6700, 0.7900, 0.8200, 0.7700, 0.9000, 0.8600, 0.5200, 0.5000, 0.7900, 0.7400, 0.7300, 0.7600, 0.7500, 0.9000, 0.5900, 0.7100, 0.8600, 0.9000, 0.7700, 0.8000, 0.8200, 0.5700, 0.7700, 0.6100, 0.8100, 0.7500, 0.6600, 0.7500, 0.6900, 0.6700, 0.8200, 0.4400, 0.8600, 0.7600, 0.7300, 0.8900, 0.6300, 0.9000, 0.8000, 0.8400, 0.5100, 0.6400, 0.5100, 0.7100, 0.9400, 0.8700, 0.5600, 0.7300, 0.6600, 0.9100, 0.8100, 0.5200, 0.8700, 0.8200, 0.8700, 0.7000, 0.8900, 0.7500, 0.7900, 0.7000, 0.5900, 0.6200, 0.8300, 0.7100, 0.9400, 0.8300, 0.7500, 0.7900, 0.4700, 0.6400, 0.5800, 0.9000, 0.9200, 0.7000, 0.7100, 0.8200, 0.6000, 0.7700, 0.9200, 0.7200, 0.6700, 0.8500, 0.7500, 0.9000, 0.8500, 0.8900, 0.8400, 0.8400, 0.6700, 0.5600, 0.9300, 0.7400, 0.6700, 0.8100, 0.5500, 0.7100, Acc_f: 0.7280, Acc_r: 0.7491
0.8800, 0.8700, 0.6700, 0.6100, 0.7100, 0.8000, 0.8400, 0.7900, 0.8600, 0.8600, 0.4800, 0.4300, 0.8300, 0.7000, 0.7300, 0.8100, 0.8000, 0.8400, 0.6800, 0.6600, 0.9000, 0.9

In [25]:
Proj_mat_lst =[]
for i in range(3):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/{arxiv_name_o}_{args.seeds[i]}.pth'))
    test_by_class(model, test_loader, i=args.unlearn_class)
    
    feature_list = []
    merged_feat_mat = []
    for batch, (x, y) in enumerate(remain_train_loader):
        x = x.cuda()
        y = y.cuda()
        mat_list = get_representation_matrix(model, x, batch_list=[256]*30)
        break
    threshold = 0.99
    merged_feat_mat = update_GPM(mat_list, threshold, merged_feat_mat)
    proj_mat = [torch.Tensor(np.dot(layer_basis, layer_basis.transpose())) for layer_basis in merged_feat_mat]
    Proj_mat_lst.append(proj_mat)

0.9100, 0.8300, 0.6100, 0.6500, 0.6700, 0.7900, 0.8200, 0.7700, 0.9000, 0.8600, 0.5200, 0.5000, 0.7900, 0.7400, 0.7300, 0.7600, 0.7500, 0.9000, 0.5900, 0.7100, 0.8600, 0.9000, 0.7700, 0.8000, 0.8200, 0.5700, 0.7700, 0.6100, 0.8100, 0.7500, 0.6600, 0.7500, 0.6900, 0.6700, 0.8200, 0.4400, 0.8600, 0.7600, 0.7300, 0.8900, 0.6300, 0.9000, 0.8000, 0.8400, 0.5100, 0.6400, 0.5100, 0.7100, 0.9400, 0.8700, 0.5600, 0.7300, 0.6600, 0.9100, 0.8100, 0.5200, 0.8700, 0.8200, 0.8700, 0.7000, 0.8900, 0.7500, 0.7900, 0.7000, 0.5900, 0.6200, 0.8300, 0.7100, 0.9400, 0.8300, 0.7500, 0.7900, 0.4700, 0.6400, 0.5800, 0.9000, 0.9200, 0.7000, 0.7100, 0.8200, 0.6000, 0.7700, 0.9200, 0.7200, 0.6700, 0.8500, 0.7500, 0.9000, 0.8500, 0.8900, 0.8400, 0.8400, 0.6700, 0.5600, 0.9300, 0.7400, 0.6700, 0.8100, 0.5500, 0.7100, Acc_f: 0.7280, Acc_r: 0.7491
Threshold:  0.99
----------------------------------------
Gradient Constraints Summary
----------------------------------------
Layer 1 : 8/27
Layer 2 : 77/576
Layer 3 : 2

In [27]:
def get_pseudo_label(args, model, x):
    masked_output = model(x)
    masked_output[:, args.unlearn_class] = -np.inf
    pseudo_labels = torch.topk(masked_output, k=1, dim=1).indices
    return pseudo_labels.reshape(-1)

for i in range(3):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/{arxiv_name_o}_{args.seeds[i]}.pth'))
    sgd_mr_model = deepcopy(model)
    test_by_class(sgd_mr_model, test_loader, i=args.unlearn_class)
    optimizer = torch.optim.SGD(sgd_mr_model.parameters(), lr=0.04)
    sgd_mr_model.train()
    for m in sgd_mr_model.modules():
        if isinstance(m, nn.BatchNorm2d) or isinstance(m, nn.BatchNorm1d):
            m.eval()

    model.eval()
    for ep in range(25):
        for batch, (x, y) in enumerate(unlearn_train_loader):
            x = x.cuda()
            y = get_pseudo_label(args, model, x)
            pred_y = sgd_mr_model(x)
            loss = criterion(pred_y, y)
            optimizer.zero_grad()
            loss.backward()
            kk = 0 
            for k, (m,params) in enumerate(sgd_mr_model.named_parameters()):
                if len(params.size())!=1:
                    sz =  params.grad.data.size(0)
                    params.grad.data = params.grad.data - torch.mm(params.grad.data.view(sz,-1),\
                                            Proj_mat_lst[i][kk].cuda()).view(params.size())
                    kk +=1
                elif len(params.size())==1:
                    params.grad.data.fill_(0)
            optimizer.step()
        print('[train] epoch {}, batch {}, loss {}'.format(ep, batch, loss))
        test_by_class(sgd_mr_model, test_loader, i=args.unlearn_class)

0.9100, 0.8300, 0.6100, 0.6500, 0.6700, 0.7900, 0.8200, 0.7700, 0.9000, 0.8600, 0.5200, 0.5000, 0.7900, 0.7400, 0.7300, 0.7600, 0.7500, 0.9000, 0.5900, 0.7100, 0.8600, 0.9000, 0.7700, 0.8000, 0.8200, 0.5700, 0.7700, 0.6100, 0.8100, 0.7500, 0.6600, 0.7500, 0.6900, 0.6700, 0.8200, 0.4400, 0.8600, 0.7600, 0.7300, 0.8900, 0.6300, 0.9000, 0.8000, 0.8400, 0.5100, 0.6400, 0.5100, 0.7100, 0.9400, 0.8700, 0.5600, 0.7300, 0.6600, 0.9100, 0.8100, 0.5200, 0.8700, 0.8200, 0.8700, 0.7000, 0.8900, 0.7500, 0.7900, 0.7000, 0.5900, 0.6200, 0.8300, 0.7100, 0.9400, 0.8300, 0.7500, 0.7900, 0.4700, 0.6400, 0.5800, 0.9000, 0.9200, 0.7000, 0.7100, 0.8200, 0.6000, 0.7700, 0.9200, 0.7200, 0.6700, 0.8500, 0.7500, 0.9000, 0.8500, 0.8900, 0.8400, 0.8400, 0.6700, 0.5600, 0.9300, 0.7400, 0.6700, 0.8100, 0.5500, 0.7100, Acc_f: 0.7280, Acc_r: 0.7491
[train] epoch 0, batch 8, loss 1.8993651866912842
0.9000, 0.8700, 0.5800, 0.7100, 0.6800, 0.7700, 0.8300, 0.7600, 0.9000, 0.8400, 0.6600, 0.4700, 0.7800, 0.7100, 0.6900, 0

# Unlearn 20 classes

In [10]:
num_unlearn = 20
args.unlearn_class = dict[num_unlearn]['unlearn_class']
arxiv_name_r = dict[num_unlearn]['arxiv_name']

arxiv_name_o = 'original_model_12-20-05-11'
Acc_r, Acc_f = np.zeros((2,3)), np.zeros((2,3))
for i in range(3):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/{arxiv_name_o}_{args.seeds[i]}.pth'))
    Acc_f[0][i], Acc_r[0][i] = test_by_class(model, test_loader, i=args.unlearn_class)

print('------------ Retrained model ------------')
for i in range(3):
    model_r = get_model(args)
    model_r.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/{arxiv_name_r}_{args.seeds[i]}.pth'))
    Acc_f[1][i], Acc_r[1][i] = test_by_class(model_r, test_loader, i=args.unlearn_class)
  

print(f'Original model Acc_f: {100*Acc_f[0].mean():.2f} \pm {100*Acc_f[0].std():.2f}')
print(f'Original model Acc_r: {100*Acc_r[0].mean():.2f} \pm {100*Acc_r[0].std():.2f}')

print(f'Retrained model Acc_f: {100*Acc_f[1].mean():.2f} \pm {100*Acc_f[1].std():.2f}')
print(f'Retrained model Acc_r: {100*Acc_r[1].mean():.2f} \pm {100*Acc_r[1].std():.2f}')


0.9100, 0.8300, 0.6100, 0.6500, 0.6700, 0.7900, 0.8200, 0.7700, 0.9000, 0.8600, 0.5200, 0.5000, 0.7900, 0.7400, 0.7300, 0.7600, 0.7500, 0.9000, 0.5900, 0.7100, 0.8600, 0.9000, 0.7700, 0.8000, 0.8200, 0.5700, 0.7700, 0.6100, 0.8100, 0.7500, 0.6600, 0.7500, 0.6900, 0.6700, 0.8200, 0.4400, 0.8600, 0.7600, 0.7300, 0.8900, 0.6300, 0.9000, 0.8000, 0.8400, 0.5100, 0.6400, 0.5100, 0.7100, 0.9400, 0.8700, 0.5600, 0.7300, 0.6600, 0.9100, 0.8100, 0.5200, 0.8700, 0.8200, 0.8700, 0.7000, 0.8900, 0.7500, 0.7900, 0.7000, 0.5900, 0.6200, 0.8300, 0.7100, 0.9400, 0.8300, 0.7500, 0.7900, 0.4700, 0.6400, 0.5800, 0.9000, 0.9200, 0.7000, 0.7100, 0.8200, 0.6000, 0.7700, 0.9200, 0.7200, 0.6700, 0.8500, 0.7500, 0.9000, 0.8500, 0.8900, 0.8400, 0.8400, 0.6700, 0.5600, 0.9300, 0.7400, 0.6700, 0.8100, 0.5500, 0.7100, Acc_f: 0.7530, Acc_r: 0.7455
0.8800, 0.8700, 0.6700, 0.6100, 0.7100, 0.8000, 0.8400, 0.7900, 0.8600, 0.8600, 0.4800, 0.4300, 0.8300, 0.7000, 0.7300, 0.8100, 0.8000, 0.8400, 0.6800, 0.6600, 0.9000, 0.9

In [11]:
Proj_mat_lst =[]
for i in range(3):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/{arxiv_name_o}_{args.seeds[i]}.pth'))
    test_by_class(model, test_loader, i=args.unlearn_class)
    
    feature_list = []
    merged_feat_mat = []
    for batch, (x, y) in enumerate(remain_train_loader):
        x = x.cuda()
        y = y.cuda()
        mat_list = get_representation_matrix(model, x, batch_list=[256]*30)
        break
    threshold = 0.99
    merged_feat_mat = update_GPM(mat_list, threshold, merged_feat_mat)
    proj_mat = [torch.Tensor(np.dot(layer_basis, layer_basis.transpose())) for layer_basis in merged_feat_mat]
    Proj_mat_lst.append(proj_mat)

0.9100, 0.8300, 0.6100, 0.6500, 0.6700, 0.7900, 0.8200, 0.7700, 0.9000, 0.8600, 0.5200, 0.5000, 0.7900, 0.7400, 0.7300, 0.7600, 0.7500, 0.9000, 0.5900, 0.7100, 0.8600, 0.9000, 0.7700, 0.8000, 0.8200, 0.5700, 0.7700, 0.6100, 0.8100, 0.7500, 0.6600, 0.7500, 0.6900, 0.6700, 0.8200, 0.4400, 0.8600, 0.7600, 0.7300, 0.8900, 0.6300, 0.9000, 0.8000, 0.8400, 0.5100, 0.6400, 0.5100, 0.7100, 0.9400, 0.8700, 0.5600, 0.7300, 0.6600, 0.9100, 0.8100, 0.5200, 0.8700, 0.8200, 0.8700, 0.7000, 0.8900, 0.7500, 0.7900, 0.7000, 0.5900, 0.6200, 0.8300, 0.7100, 0.9400, 0.8300, 0.7500, 0.7900, 0.4700, 0.6400, 0.5800, 0.9000, 0.9200, 0.7000, 0.7100, 0.8200, 0.6000, 0.7700, 0.9200, 0.7200, 0.6700, 0.8500, 0.7500, 0.9000, 0.8500, 0.8900, 0.8400, 0.8400, 0.6700, 0.5600, 0.9300, 0.7400, 0.6700, 0.8100, 0.5500, 0.7100, Acc_f: 0.7530, Acc_r: 0.7455
Threshold:  0.99
----------------------------------------
Gradient Constraints Summary
----------------------------------------
Layer 1 : 9/27
Layer 2 : 77/576
Layer 3 : 2

In [ ]:
def get_pseudo_label(args, model, x):
    masked_output = model(x)
    masked_output[:, args.unlearn_class] = -np.inf
    pseudo_labels = torch.topk(masked_output, k=1, dim=1).indices
    return pseudo_labels.reshape(-1)

for i in range(3):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/{arxiv_name_o}_{args.seeds[i]}.pth'))
    sgd_mr_model = deepcopy(model)
    test_by_class(sgd_mr_model, test_loader, i=args.unlearn_class)
    optimizer = torch.optim.SGD(sgd_mr_model.parameters(), lr=0.56)
    sgd_mr_model.train()
    for m in sgd_mr_model.modules():
        if isinstance(m, nn.BatchNorm2d) or isinstance(m, nn.BatchNorm1d):
            m.eval()

    model.eval()
    for ep in range(25):
        for batch, (x, y) in enumerate(unlearn_train_loader):
            x = x.cuda()
            y = get_pseudo_label(args, model, x)
            pred_y = sgd_mr_model(x)
            loss = criterion(pred_y, y)
            optimizer.zero_grad()
            loss.backward()
            kk = 0 
            for k, (m,params) in enumerate(sgd_mr_model.named_parameters()):
                if len(params.size())!=1:
                    sz =  params.grad.data.size(0)
                    params.grad.data = params.grad.data - torch.mm(params.grad.data.view(sz,-1),\
                                            Proj_mat_lst[i][kk].cuda()).view(params.size())
                    kk +=1
                elif len(params.size())==1:
                    params.grad.data.fill_(0)
            optimizer.step()
        print('[train] epoch {}, batch {}, loss {}'.format(ep, batch, loss))
        test_by_class(sgd_mr_model, test_loader, i=args.unlearn_class)

In [101]:
Acc_r = 100*np.array([0.9245, 0.9305, 0.9270])
Acc_f = 100*np.array([0.0020, 0.0060, 0.0100])

print(f'Mince Acc_f: {Acc_f.mean():.2f} \pm {Acc_f.std():.2f}')
print(f'Mince Acc_r: {Acc_r.mean():.2f} \pm {Acc_r.std():.2f}')

Mince Acc_f: 0.60 \pm 0.33
Mince Acc_r: 92.73 \pm 0.25


# Unlearn 3 classes


In [89]:
arxiv_name = dict[3]['arxiv_name']
args.unlearn_class = dict[3]['unlearn_class']
Acc_r, Acc_f = np.zeros((2,3)), np.zeros((2,3))
for i in range(3):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/original_model_12-20-02-15_{args.seeds[i]}.pth'))
    Acc_f[0][i], Acc_r[0][i] = test_by_class(model, test_loader, i=args.unlearn_class)

print('------------ Retrained model ------------')
for i in range(3):
    model_r = get_model(args)
    try:
        model_r.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/retrain_model_{arxiv_name}_{args.seeds[i]}.pth'))
        Acc_f[1][i], Acc_r[1][i] = test_by_class(model_r, test_loader, i=args.unlearn_class)
    except:
        print('No retrained model')
        break

print(f'Original model Acc_f: {100*Acc_f[0].mean():.2f} \pm {100*Acc_f[0].std():.2f}')
print(f'Original model Acc_r: {100*Acc_r[0].mean():.2f} \pm {100*Acc_r[0].std():.2f}')

print(f'Retrained model Acc_f: {100*Acc_f[1].mean():.2f} \pm {100*Acc_f[1].std():.2f}')
print(f'Retrained model Acc_r: {100*Acc_r[1].mean():.2f} \pm {100*Acc_r[1].std():.2f}')

arxiv_name = 'original_model_12-20-02-15'

0.9460, 0.9610, 0.8810, 0.7980, 0.9060, 0.8590, 0.9580, 0.8990, 0.9420, 0.9070, Acc_f: 0.8727, Acc_r: 0.9199
0.9250, 0.9540, 0.8760, 0.8210, 0.9110, 0.8740, 0.9270, 0.9390, 0.9370, 0.9420, Acc_f: 0.8830, Acc_r: 0.9224
0.9270, 0.9530, 0.8860, 0.7790, 0.9280, 0.8710, 0.9290, 0.9280, 0.9530, 0.9470, Acc_f: 0.8677, Acc_r: 0.9283
------------ Retrained model ------------
0.9350, 0.0000, 0.8920, 0.0000, 0.9370, 0.0000, 0.9640, 0.9460, 0.9480, 0.9730, Acc_f: 0.0000, Acc_r: 0.9421
0.9300, 0.0000, 0.8990, 0.0000, 0.9490, 0.0000, 0.9450, 0.9560, 0.9450, 0.9590, Acc_f: 0.0000, Acc_r: 0.9404
0.9420, 0.0000, 0.9120, 0.0000, 0.9570, 0.0000, 0.9590, 0.9390, 0.9390, 0.9640, Acc_f: 0.0000, Acc_r: 0.9446
Original model Acc_f: 87.44 \pm 0.64
Original model Acc_r: 92.35 \pm 0.35
Retrained model Acc_f: 0.00 \pm 0.00
Retrained model Acc_r: 94.24 \pm 0.17


In [91]:
def get_pseudo_label(args, model, x):
    masked_output = model(x)
    masked_output[:, args.unlearn_class] = -np.inf
    pseudo_labels = torch.topk(masked_output, k=1, dim=1).indices
    return pseudo_labels.reshape(-1)

for i in range(3):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/{arxiv_name}_{args.seeds[i]}.pth'))
    sgd_mr_model = deepcopy(model)
    test_by_class(sgd_mr_model, test_loader, i=args.unlearn_class)
    optimizer = torch.optim.SGD(sgd_mr_model.parameters(), lr=0.04)
    sgd_mr_model.train()
    for m in sgd_mr_model.modules():
        if isinstance(m, nn.BatchNorm2d) or isinstance(m, nn.BatchNorm1d):
            m.eval()

    model.eval()
    for ep in range(25):
        for batch, (x, y) in enumerate(unlearn_train_loader):
            x = x.cuda()
            y = get_pseudo_label(args, model, x)
            pred_y = sgd_mr_model(x)
            loss = criterion(pred_y, y)
            optimizer.zero_grad()
            loss.backward()
            kk = 0 
            for k, (m,params) in enumerate(sgd_mr_model.named_parameters()):
                if len(params.size())!=1:
                    sz =  params.grad.data.size(0)
                    params.grad.data = params.grad.data - torch.mm(params.grad.data.view(sz,-1),\
                                            Proj_mat_lst[i][kk].cuda()).view(params.size())
                    kk +=1
                elif len(params.size())==1:
                    params.grad.data.fill_(0)
            optimizer.step()
        print('[train] epoch {}, batch {}, loss {}'.format(ep, batch, loss))
        test_by_class(sgd_mr_model, test_loader, i=args.unlearn_class)

0.9460, 0.9610, 0.8810, 0.7980, 0.9060, 0.8590, 0.9580, 0.8990, 0.9420, 0.9070, Acc_f: 0.8727, Acc_r: 0.9199
[train] epoch 0, batch 26, loss 1.7692084312438965
0.8860, 0.4480, 0.8700, 0.1550, 0.9100, 0.0760, 0.9480, 0.8980, 0.9690, 0.9470, Acc_f: 0.2263, Acc_r: 0.9183
[train] epoch 1, batch 26, loss 1.0091466903686523
0.9080, 0.2070, 0.8830, 0.1140, 0.9200, 0.0530, 0.9590, 0.9200, 0.9630, 0.9430, Acc_f: 0.1247, Acc_r: 0.9280
[train] epoch 2, batch 26, loss 0.6982631087303162
0.9220, 0.1080, 0.8830, 0.0890, 0.9120, 0.0380, 0.9680, 0.9170, 0.9620, 0.9390, Acc_f: 0.0783, Acc_r: 0.9290
[train] epoch 3, batch 26, loss 0.5673158764839172
0.9350, 0.0560, 0.8860, 0.0800, 0.9240, 0.0340, 0.9670, 0.9240, 0.9590, 0.9400, Acc_f: 0.0567, Acc_r: 0.9336
[train] epoch 4, batch 26, loss 0.5916770100593567
0.9430, 0.0360, 0.8880, 0.0580, 0.9180, 0.0270, 0.9680, 0.9290, 0.9540, 0.9360, Acc_f: 0.0403, Acc_r: 0.9337
[train] epoch 5, batch 26, loss 0.5266324281692505
0.9480, 0.0200, 0.8870, 0.0500, 0.9090, 

In [92]:
Acc_r = 100*np.array([0.9369,0.9430, 0.9354])
Acc_f = 100*np.array([0.0043,0.0073,0.0150])

print(f'Mince Acc_f: {Acc_f.mean():.2f} \pm {Acc_f.std():.2f}')
print(f'Mince Acc_r: {Acc_r.mean():.2f} \pm {Acc_r.std():.2f}')

Mince Acc_f: 0.89 \pm 0.45
Mince Acc_r: 93.84 \pm 0.33


In [85]:
arxiv_name = dict[4]['arxiv_name']
args.unlearn_class = dict[4]['unlearn_class']
Acc_r, Acc_f = np.zeros((2,3)), np.zeros((2,3))
for i in range(3):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/original_model_12-20-02-15_{args.seeds[i]}.pth'))
    Acc_f[0][i], Acc_r[0][i] = test_by_class(model, test_loader, i=args.unlearn_class)

print('------------ Retrained model ------------')
for i in range(3):
    model_r = get_model(args)
    try:
        model_r.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/retrain_model_{arxiv_name}_{args.seeds[i]}.pth'))
        Acc_f[1][i], Acc_r[1][i] = test_by_class(model_r, test_loader, i=args.unlearn_class)
    except:
        print('No retrained model')
        break

print(f'Original model Acc_f: {100*Acc_f[0].mean():.2f} \pm {100*Acc_f[0].std():.2f}')
print(f'Original model Acc_r: {100*Acc_r[0].mean():.2f} \pm {100*Acc_r[0].std():.2f}')

print(f'Retrained model Acc_f: {100*Acc_f[1].mean():.2f} \pm {100*Acc_f[1].std():.2f}')
print(f'Retrained model Acc_r: {100*Acc_r[1].mean():.2f} \pm {100*Acc_r[1].std():.2f}')

arxiv_name = 'original_model_12-20-02-15'

0.9460, 0.9610, 0.8810, 0.7980, 0.9060, 0.8590, 0.9580, 0.8990, 0.9420, 0.9070, Acc_f: 0.8792, Acc_r: 0.9233
0.9250, 0.9540, 0.8760, 0.8210, 0.9110, 0.8740, 0.9270, 0.9390, 0.9370, 0.9420, Acc_f: 0.8970, Acc_r: 0.9197
0.9270, 0.9530, 0.8860, 0.7790, 0.9280, 0.8710, 0.9290, 0.9280, 0.9530, 0.9470, Acc_f: 0.8827, Acc_r: 0.9283
------------ Retrained model ------------
0.9430, 0.0000, 0.9100, 0.0000, 0.9560, 0.0000, 0.9540, 0.0000, 0.9560, 0.9680, Acc_f: 0.0000, Acc_r: 0.9478
0.9210, 0.0000, 0.8900, 0.0000, 0.9400, 0.0000, 0.9520, 0.0000, 0.9580, 0.9760, Acc_f: 0.0000, Acc_r: 0.9395
0.9270, 0.0000, 0.9130, 0.0000, 0.9380, 0.0000, 0.9520, 0.0000, 0.9580, 0.9580, Acc_f: 0.0000, Acc_r: 0.9410
Original model Acc_f: 88.63 \pm 0.77
Original model Acc_r: 92.38 \pm 0.36
Retrained model Acc_f: 0.00 \pm 0.00
Retrained model Acc_r: 94.28 \pm 0.36


In [86]:
unlearn_indices = np.where(np.isin(train_targets_list, args.unlearn_class))[0]

# conver to the original indices
unlearn_indices = train_loader.sampler.indices[unlearn_indices]

unlearn_sampler = torch.utils.data.SubsetRandomSampler(unlearn_indices)
unlearn_subset_loader = torch.utils.data.DataLoader(train_loader.dataset, 
                                                    batch_size=args.batch_size, 
                                                    sampler=unlearn_sampler)
remain_class = np.setdiff1d(np.arange(args.num_classes), args.unlearn_class)

remain_indices = np.where(~np.isin(train_targets_list, args.unlearn_class))[0]
remain_indices = train_loader.sampler.indices[remain_indices]

remain_sampler = torch.utils.data.SubsetRandomSampler(remain_indices)
remain_loader = torch.utils.data.DataLoader(train_loader.dataset, 
                                            batch_size=args.batch_size, 
                                            sampler=remain_sampler)

Proj_mat_lst =[]
for i in range(3):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/{arxiv_name}_{args.seeds[i]}.pth'))
    print(f'------------ Trail {i} ------------')
    merged_feat_mat = []
    for cls_id in range(10): 
        cls_indices = np.where(np.isin(train_targets_list, cls_id))[0]
        cls_indices = train_loader.sampler.indices[cls_indices]
        cls_sampler = torch.utils.data.SubsetRandomSampler(cls_indices)
        cls_loader_dict = torch.utils.data.DataLoader(train_loader.dataset, 
                                                                batch_size=args.batch_size, 
                                                                sampler=cls_sampler)
        if cls_id in args.unlearn_class:
            continue
        for batch, (x, y) in enumerate(cls_loader_dict ):
            x = x.cuda()
            y = y.cuda()
            mat_list = get_representation_matrix(model, 
                                                x, 
                                                batch_list=[24, 100, 100, 125, 125, 250, 250, 256, 256])
            break
        threshold = 0.97 + 0.003*cls_id
        merged_feat_mat = update_GPM(mat_list, threshold, merged_feat_mat)
        proj_mat = [torch.Tensor(np.dot(layer_basis, layer_basis.transpose())) for layer_basis in merged_feat_mat]
        Proj_mat_lst.append(proj_mat)


------------ Trail 0 ------------
Threshold:  0.97
----------------------------------------
Gradient Constraints Summary
----------------------------------------
Layer 1 : 5/27
Layer 2 : 49/432
Layer 3 : 183/432
Layer 4 : 555/864
Layer 5 : 619/864
Layer 6 : 621/864
Layer 7 : 665/864
Layer 8 : 75/96
Layer 9 : 4/96
----------------------------------------
Threshold:  0.976
----------------------------------------
Gradient Constraints Summary
----------------------------------------
Layer 1 : 7/27
Layer 2 : 60/432
Layer 3 : 215/432
Layer 4 : 644/864
Layer 5 : 690/864
Layer 6 : 678/864
Layer 7 : 756/864
Layer 8 : 88/96
Layer 9 : 8/96
----------------------------------------
Threshold:  0.982
----------------------------------------
Gradient Constraints Summary
----------------------------------------
Layer 1 : 8/27
Layer 2 : 70/432
Layer 3 : 244/432
Layer 4 : 708/864
Layer 5 : 739/864
Layer 6 : 721/864
Layer 7 : 788/864
Layer 8 : 92/96
Layer 9 : 10/96
--------------------------------------

In [87]:
def get_pseudo_label(args, model, x):
    masked_output = model(x)
    masked_output[:, args.unlearn_class] = -np.inf
    pseudo_labels = torch.topk(masked_output, k=1, dim=1).indices
    return pseudo_labels.reshape(-1)

for i in range(3):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/{arxiv_name}_{args.seeds[i]}.pth'))
    sgd_mr_model = deepcopy(model)
    test_by_class(sgd_mr_model, test_loader, i=args.unlearn_class)
    optimizer = torch.optim.SGD(sgd_mr_model.parameters(), lr=0.04)
    sgd_mr_model.train()
    for m in sgd_mr_model.modules():
        if isinstance(m, nn.BatchNorm2d) or isinstance(m, nn.BatchNorm1d):
            m.eval()

    model.eval()
    for ep in range(25):
        for batch, (x, y) in enumerate(unlearn_train_loader:
            x = x.cuda()
            y = get_pseudo_label(args, model, x)
            pred_y = sgd_mr_model(x)
            loss = criterion(pred_y, y)
            optimizer.zero_grad()
            loss.backward()
            kk = 0 
            for k, (m,params) in enumerate(sgd_mr_model.named_parameters()):
                if len(params.size())!=1:
                    sz =  params.grad.data.size(0)
                    params.grad.data = params.grad.data - torch.mm(params.grad.data.view(sz,-1),\
                                            Proj_mat_lst[i][kk].cuda()).view(params.size())
                    kk +=1
                elif len(params.size())==1:
                    params.grad.data.fill_(0)
            optimizer.step()
        print('[train] epoch {}, batch {}, loss {}'.format(ep, batch, loss))
        test_by_class(sgd_mr_model, test_loader, i=args.unlearn_class)

0.9460, 0.9610, 0.8810, 0.7980, 0.9060, 0.8590, 0.9580, 0.8990, 0.9420, 0.9070, Acc_f: 0.8792, Acc_r: 0.9233
[train] epoch 0, batch 35, loss 1.23568594455719
0.9010, 0.4310, 0.8760, 0.1330, 0.9190, 0.0530, 0.9720, 0.0800, 0.9650, 0.9410, Acc_f: 0.1743, Acc_r: 0.9290
[train] epoch 1, batch 35, loss 0.7198920845985413
0.9180, 0.1840, 0.8850, 0.0890, 0.9280, 0.0430, 0.9690, 0.0460, 0.9610, 0.9410, Acc_f: 0.0905, Acc_r: 0.9337
[train] epoch 2, batch 35, loss 0.6311068534851074
0.9360, 0.1010, 0.8830, 0.0660, 0.9220, 0.0320, 0.9740, 0.0310, 0.9560, 0.9400, Acc_f: 0.0575, Acc_r: 0.9352
[train] epoch 3, batch 35, loss 0.7038785815238953
0.9410, 0.0540, 0.8890, 0.0450, 0.9190, 0.0210, 0.9730, 0.0220, 0.9520, 0.9410, Acc_f: 0.0355, Acc_r: 0.9358
[train] epoch 4, batch 35, loss 0.6717749834060669
0.9430, 0.0340, 0.8840, 0.0390, 0.9270, 0.0170, 0.9710, 0.0190, 0.9490, 0.9410, Acc_f: 0.0273, Acc_r: 0.9358
[train] epoch 5, batch 35, loss 0.33119139075279236
0.9420, 0.0210, 0.8800, 0.0320, 0.9240, 0

In [88]:
Acc_r = 100*np.array([0.9395, 0.9437, 0.9392])
Acc_f = 100*np.array([0.0003,  0.0030, 0.0060])

print(f'Mince Acc_f: {Acc_f.mean():.2f} \pm {Acc_f.std():.2f}')
print(f'Mince Acc_r: {Acc_r.mean():.2f} \pm {Acc_r.std():.2f}')

Mince Acc_f: 0.31 \pm 0.23
Mince Acc_r: 94.08 \pm 0.21


# Unlearn 5 classes

In [78]:
arxiv_name = dict[5]['arxiv_name']
args.unlearn_class = dict[5]['unlearn_class']
Acc_r, Acc_f = np.zeros((2,3)), np.zeros((2,3))
for i in range(3):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/original_model_12-20-02-15_{args.seeds[i]}.pth'))
    Acc_f[0][i], Acc_r[0][i] = test_by_class(model, test_loader, i=args.unlearn_class)

print('------------ Retrained model ------------')
for i in range(3):
    model_r = get_model(args)
    try:
        model_r.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/retrain_model_{arxiv_name}_{args.seeds[i]}.pth'))
        Acc_f[1][i], Acc_r[1][i] = test_by_class(model_r, test_loader, i=args.unlearn_class)
    except:
        print('No retrained model')
        break

print(f'Original model Acc_f: {100*Acc_f[0].mean():.2f} \pm {100*Acc_f[0].std():.2f}')
print(f'Original model Acc_r: {100*Acc_r[0].mean():.2f} \pm {100*Acc_r[0].std():.2f}')

print(f'Retrained model Acc_f: {100*Acc_f[1].mean():.2f} \pm {100*Acc_f[1].std():.2f}')
print(f'Retrained model Acc_r: {100*Acc_r[1].mean():.2f} \pm {100*Acc_r[1].std():.2f}')

arxiv_name = 'original_model_12-20-02-15'

0.9460, 0.9610, 0.8810, 0.7980, 0.9060, 0.8590, 0.9580, 0.8990, 0.9420, 0.9070, Acc_f: 0.8796, Acc_r: 0.9318
0.9250, 0.9540, 0.8760, 0.8210, 0.9110, 0.8740, 0.9270, 0.9390, 0.9370, 0.9420, Acc_f: 0.8928, Acc_r: 0.9284
0.9270, 0.9530, 0.8860, 0.7790, 0.9280, 0.8710, 0.9290, 0.9280, 0.9530, 0.9470, Acc_f: 0.8834, Acc_r: 0.9368
------------ Retrained model ------------
0.9450, 0.0000, 0.0000, 0.0000, 0.9620, 0.0000, 0.9620, 0.0000, 0.9610, 0.9660, Acc_f: 0.0000, Acc_r: 0.9592
0.9530, 0.0000, 0.0000, 0.0000, 0.9710, 0.0000, 0.9640, 0.0000, 0.9520, 0.9810, Acc_f: 0.0000, Acc_r: 0.9642
0.9550, 0.0000, 0.0000, 0.0000, 0.9600, 0.0000, 0.9650, 0.0000, 0.9580, 0.9630, Acc_f: 0.0000, Acc_r: 0.9602
Original model Acc_f: 88.53 \pm 0.55
Original model Acc_r: 93.23 \pm 0.34
Retrained model Acc_f: 0.00 \pm 0.00
Retrained model Acc_r: 96.12 \pm 0.22


In [79]:
unlearn_indices = np.where(np.isin(train_targets_list, args.unlearn_class))[0]

# conver to the original indices
unlearn_indices = train_loader.sampler.indices[unlearn_indices]

unlearn_sampler = torch.utils.data.SubsetRandomSampler(unlearn_indices)
unlearn_subset_loader = torch.utils.data.DataLoader(train_loader.dataset, 
                                                    batch_size=args.batch_size, 
                                                    sampler=unlearn_sampler)
remain_class = np.setdiff1d(np.arange(args.num_classes), args.unlearn_class)

remain_indices = np.where(~np.isin(train_targets_list, args.unlearn_class))[0]
remain_indices = train_loader.sampler.indices[remain_indices]

remain_sampler = torch.utils.data.SubsetRandomSampler(remain_indices)
remain_loader = torch.utils.data.DataLoader(train_loader.dataset, 
                                            batch_size=args.batch_size, 
                                            sampler=remain_sampler)

Proj_mat_lst =[]
for i in range(3):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/{arxiv_name}_{args.seeds[i]}.pth'))
    print(f'------------ Trail {i} ------------')
    merged_feat_mat = []
    for cls_id in range(10): 
        cls_indices = np.where(np.isin(train_targets_list, cls_id))[0]
        cls_indices = train_loader.sampler.indices[cls_indices]
        cls_sampler = torch.utils.data.SubsetRandomSampler(cls_indices)
        cls_loader_dict = torch.utils.data.DataLoader(train_loader.dataset, 
                                                                batch_size=args.batch_size, 
                                                                sampler=cls_sampler)
        if cls_id in args.unlearn_class:
            continue
        for batch, (x, y) in enumerate(cls_loader_dict ):
            x = x.cuda()
            y = y.cuda()
            mat_list = get_representation_matrix(model, 
                                                x, 
                                                batch_list=[24, 100, 100, 125, 125, 250, 250, 256, 256])
            break
        threshold = 0.97 + 0.003*cls_id
        merged_feat_mat = update_GPM(mat_list, threshold, merged_feat_mat)
        proj_mat = [torch.Tensor(np.dot(layer_basis, layer_basis.transpose())) for layer_basis in merged_feat_mat]
        Proj_mat_lst.append(proj_mat)


------------ Trail 0 ------------
Threshold:  0.97
----------------------------------------
Gradient Constraints Summary
----------------------------------------
Layer 1 : 5/27
Layer 2 : 48/432
Layer 3 : 180/432
Layer 4 : 552/864
Layer 5 : 618/864
Layer 6 : 617/864
Layer 7 : 666/864
Layer 8 : 76/96
Layer 9 : 4/96
----------------------------------------
Threshold:  0.982
----------------------------------------
Gradient Constraints Summary
----------------------------------------
Layer 1 : 8/27
Layer 2 : 70/432
Layer 3 : 240/432
Layer 4 : 700/864
Layer 5 : 733/864
Layer 6 : 716/864
Layer 7 : 779/864
Layer 8 : 90/96
Layer 9 : 8/96
----------------------------------------
Threshold:  0.988
----------------------------------------
Gradient Constraints Summary
----------------------------------------
Layer 1 : 11/27
Layer 2 : 107/432
Layer 3 : 288/432
Layer 4 : 758/864
Layer 5 : 779/864
Layer 6 : 761/864
Layer 7 : 809/864
Layer 8 : 93/96
Layer 9 : 10/96
------------------------------------

In [83]:
def get_pseudo_label(args, model, x):
    masked_output = model(x)
    masked_output[:, args.unlearn_class] = -np.inf
    pseudo_labels = torch.topk(masked_output, k=1, dim=1).indices
    return pseudo_labels.reshape(-1)

for i in range(3):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/{arxiv_name}_{args.seeds[i]}.pth'))
    sgd_mr_model = deepcopy(model)
    test_by_class(sgd_mr_model, test_loader, i=args.unlearn_class)
    optimizer = torch.optim.SGD(sgd_mr_model.parameters(), lr=0.04)
    sgd_mr_model.train()
    for m in sgd_mr_model.modules():
        if isinstance(m, nn.BatchNorm2d) or isinstance(m, nn.BatchNorm1d):
            m.eval()

    model.eval()
    for ep in range(25):
        for batch, (x, y) in enumerate(unlearn_train_loader:
            x = x.cuda()
            y = get_pseudo_label(args, model, x)
            pred_y = sgd_mr_model(x)
            loss = criterion(pred_y, y)
            optimizer.zero_grad()
            loss.backward()
            kk = 0 
            for k, (m,params) in enumerate(sgd_mr_model.named_parameters()):
                if len(params.size())!=1:
                    sz =  params.grad.data.size(0)
                    params.grad.data = params.grad.data - torch.mm(params.grad.data.view(sz,-1),\
                                            Proj_mat_lst[i][kk].cuda()).view(params.size())
                    kk +=1
                elif len(params.size())==1:
                    params.grad.data.fill_(0)
            optimizer.step()
        print('[train] epoch {}, batch {}, loss {}'.format(ep, batch, loss))
        test_by_class(sgd_mr_model, test_loader, i=args.unlearn_class)

0.9460, 0.9610, 0.8810, 0.7980, 0.9060, 0.8590, 0.9580, 0.8990, 0.9420, 0.9070, Acc_f: 0.8796, Acc_r: 0.9318
[train] epoch 0, batch 43, loss 1.216407299041748
0.8700, 0.2700, 0.2570, 0.0780, 0.9180, 0.0200, 0.9700, 0.0490, 0.9660, 0.9230, Acc_f: 0.1348, Acc_r: 0.9294
[train] epoch 1, batch 43, loss 0.9499720931053162
0.9100, 0.1340, 0.1460, 0.0580, 0.9240, 0.0210, 0.9750, 0.0210, 0.9620, 0.9420, Acc_f: 0.0760, Acc_r: 0.9426
[train] epoch 2, batch 43, loss 0.6702789664268494
0.9290, 0.0680, 0.1140, 0.0500, 0.9270, 0.0210, 0.9790, 0.0160, 0.9580, 0.9430, Acc_f: 0.0538, Acc_r: 0.9472
[train] epoch 3, batch 43, loss 0.6160269379615784
0.9430, 0.0490, 0.0950, 0.0400, 0.9310, 0.0210, 0.9790, 0.0130, 0.9500, 0.9430, Acc_f: 0.0436, Acc_r: 0.9492
[train] epoch 4, batch 43, loss 0.56300950050354
0.9500, 0.0360, 0.0800, 0.0320, 0.9220, 0.0180, 0.9810, 0.0130, 0.9500, 0.9480, Acc_f: 0.0358, Acc_r: 0.9502
[train] epoch 5, batch 43, loss 0.4867788255214691
0.9540, 0.0260, 0.0740, 0.0260, 0.9290, 0.0

In [84]:
Acc_r = 100*np.array([0.9600, 0.9632, 0.9614])
Acc_f = 100*np.array([0.0012, 0.0054, 0.0138])

print(f'Mince Acc_f: {Acc_f.mean():.2f} \pm {Acc_f.std():.2f}')
print(f'Mince Acc_r: {Acc_r.mean():.2f} \pm {Acc_r.std():.2f}')

Mince Acc_f: 0.68 \pm 0.52
Mince Acc_r: 96.15 \pm 0.13
